In [5]:
from random import sample
import random
from tqdm import tqdm
import my_utils as mu
import pandas as pd
import requests
tqdm.pandas()

- gold 구간
- 첫 df gameId, match, timeline

In [6]:
tier_lst = ['I','II','III','IV']

In [7]:
lst = []
p = random.randrange(1,10)
for t in tqdm(tier_lst):
    url='https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/GOLD/'+t+'?page='+str(p)+'&api_key='+mu.riot_api_key
    res=requests.get(url).json()
    lst = lst+sample(res, 5)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.78it/s]


In [8]:
len(lst)

20

In [9]:
summonerName_lst = list(map(lambda x: x['summonerName'], lst))
summonerName_lst

['말년에오엘',
 '보그다노비치',
 '쾌 또 ',
 '빨리찾지마',
 '13억',
 '국밥오징어젓갈',
 '투 척',
 '아리 꼬리털 한가닥',
 'lllIIlllIlI',
 '쉬 이 바',
 '앞구르기후뒷점멸',
 '방패용사의성공담',
 '세브란스 공주',
 '정치하면 CS통제',
 'lilliililllilil',
 '비션갱 창모',
 '오른누나아칼리',
 'bjhun1',
 '꼰대 할아버지',
 'hibit']

In [10]:
puuid_lst = []
for n in tqdm(summonerName_lst):
    try:
        puuid_lst.append(mu.get_puuid(n))
    except:
        continue

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:06<00:00,  3.31it/s]


In [11]:
len(puuid_lst)

20

In [12]:
matchId_lst = []
for p in tqdm(puuid_lst):
    matchId_lst = matchId_lst + mu.get_matchId(p, 2)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:05<00:00,  3.63it/s]


In [13]:
len(matchId_lst)

40

In [14]:
import time

In [15]:
time.sleep(5)
print('test')

test


In [16]:
tmp = []
for m_id in tqdm(matchId_lst):
    tmp_lst = []
    tmp_lst.append(m_id)
    matches, timelines = mu.get_matches_timelines(m_id)
    tmp_lst.append(matches)
    tmp_lst.append(timelines)
    tmp.append(tmp_lst)

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:38<00:00,  1.04it/s]


In [17]:
len(tmp)

40

In [18]:
df = pd.DataFrame(tmp, columns = ['gameId','match','timeline'])
df.head()

,gameId,match,timeline
0,KR_5537052574,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
1,KR_5526042196,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
2,KR_5405682235,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
3,KR_5405598379,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
4,KR_5414479127,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."


## 데이터 수집 함수화
- 1) gameId, match, timeline 으로 구성된 원시데이터 df 만드는 함수 (인자값으로 티어 ex.gold silver )
- 2) 원시데이터 df를 input 넣어서 output으로 match_df, timeline_df (인자값이 원시데이터 df)
- match_df ['gameId','gameDuration','gameVersion','summonerName','summonerLevel','participantId','championName',
  'champExperience','teamPosition','teamId','win','kills','deaths','assists',
  'totalDamageDealtToChampions','totalDamageTaken']
- timeline_df ['gameId','participantId','g5','g6' ~ 'g25']
- 3) db에 insert (인자값이 전처리를 진행한 match_df, timeline_df)
- match_pk, timeline_pk = gameId, participantId
- insert 하기전에 dataframe 중복제거
- match_df, timeline_df 컬럼값 임의로 추천해드린것 - 추후 프로젝트에 개인적으로 필요한 데이터가 있을시 추가해서 수집 가능

In [8]:
riot_api_key = 'RGAPI-b7be75a2-cbe2-4837-9ec5-d66317ee9750'

In [9]:
def get_rawData(tier):
    tier_lst = ['I','II','III','IV']
    lst = []
    p = random.randrange(1,10)
    
    # riot api
    for t in tqdm(tier_lst):
        url='https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/'+tier+'/'+t+'?page='+str(p)+'&api_key='+riot_api_key
        res = requests.get(url).json()
        lst = lst+sample(res, 5)
    
    # summonerName
    print('get summonerName...')
    summonerName_lst = list(map(lambda x: x['summonerName'], lst))
    
    # puuid
    print('get puuid...')
    puuid_lst = []
    for n in tqdm(summonerName_lst):
        try:
            puuid_lst.append(mu.get_puuid(n))
        except:
            continue
            
    # matchId
    print('get matchId...')
    matchId_lst = []
    for p in tqdm(puuid_lst):
        matchId_lst = matchId_lst+mu.get_matchId(p, 2)
        
    # match, timeline
    print('get match & timeline...')
    tmp = []
    for m_id in tqdm(matchId_lst):
        tmp_lst = []
        tmp_lst.append(m_id)
        matches, timelines = mu.get_matches_timelines(m_id)
        tmp_lst.append(matches)
        tmp_lst.append(timelines)
        tmp.append(tmp_lst)
    
    df = pd.DataFrame(tmp, columns = ['gameId','match','timeline'])
    print('complete!')
    return df

In [10]:
tier = 'GOLD'
raw_data = get_rawData(tier)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

get summonerName...
get puuid...


  0%|                                                                                           | 0/19 [00:00<?, ?it/s]

get matchId...


  0%|                                                                                           | 0/38 [00:00<?, ?it/s]

get match & timeline...


100%|██████████████████████████████████████████████████████████████████████████████████| 38/38 [00:38<00:00,  1.02s/it]

complete!


In [29]:
raw_data.iloc[0].match['info']['gameMode']

'CLASSIC'

In [11]:
def get_match_timeline_df(df):
    print('get matches & timelines...')
    match_lst = []
    timeline_lst = []
    for i in tqdm(range(len(df))):
        if df.iloc[i].match['info']['gameMode'] == 'CLASSIC':
            # matches
            for j in range(10):
                tmp = []
                tmp.append(df.iloc[i].gameId)
                tmp.append(df.iloc[i].match['info']['gameDuration'])
                tmp.append(df.iloc[i].match['info']['gameVersion'])
                tmp.append(df.iloc[i].match['info']['participants'][j]['summonerName'])
                tmp.append(df.iloc[i].match['info']['participants'][j]['summonerLevel'])
                tmp.append(df.iloc[i].match['info']['participants'][j]['participantId'])
                tmp.append(df.iloc[i].match['info']['participants'][j]['championName'])
                tmp.append(df.iloc[i].match['info']['participants'][j]['champExperience'])
                tmp.append(df.iloc[i].match['info']['participants'][j]['teamPosition'])
                tmp.append(df.iloc[i].match['info']['participants'][j]['teamId'])
                tmp.append(df.iloc[i].match['info']['participants'][j]['win'])
                tmp.append(df.iloc[i].match['info']['participants'][j]['kills'])
                tmp.append(df.iloc[i].match['info']['participants'][j]['deaths'])
                tmp.append(df.iloc[i].match['info']['participants'][j]['assists'])
                tmp.append(df.iloc[i].match['info']['participants'][j]['totalDamageDealtToChampions'])
                tmp.append(df.iloc[i].match['info']['participants'][j]['totalDamageTaken'])
                match_lst.append(tmp)
                
            # timelines
            for j in range(1, 11):
                tmp = []
                tmp.append(df.iloc[i].gameId)
                tmp.append(df.iloc[i].timeline['info']['frames'][0]['participantFrames'][str(j)]['participantId'])
                for k in range(5, 26):
                    try:
                        tmp.append(df.iloc[i].timeline['info']['frames'][k]['participantFrames'][str(j)]['totalGold'])
                    except:
                        tmp.append(0)
                timeline_lst.append(tmp)
            
    col = ['gameId','gameDuration','gameVersion','summonerName','summonerLevel','participantId','championName','champExperience',
           'teamPosition','teamId','win','kills','deaths','assists','totalDamageDealtToChampions','totalDamageTaken']
    m_df = pd.DataFrame(match_lst, columns=col).drop_duplicates()

    col = ['gameId','participantId','g_5','g_6','g_7','g_8','g_9','g_10','g_11','g_12','g_13','g_14','g_15','g_16','g_17',
           'g_18','g_19','g_20','g_21','g_22','g_23','g_24','g_25']
    t_df = pd.DataFrame(timeline_lst, columns=col).drop_duplicates()
    
    print('match_df length: ',len(m_df))
    print('timeline_df length: ',len(t_df))
    print('complete!')
    return m_df, t_df

In [12]:
match_df, timeline_df = get_match_timeline_df(raw_data)

 13%|██████████▉                                                                        | 5/38 [00:00<00:00, 43.79it/s]

get matches & timelines...


100%|██████████████████████████████████████████████████████████████████████████████████| 38/38 [00:01<00:00, 34.43it/s]

match_df length:  320
timeline_df length:  320
complete!


In [13]:
match_df

,gameId,gameDuration,gameVersion,summonerName,summonerLevel,participantId,championName,champExperience,teamPosition,teamId,win,kills,deaths,assists,totalDamageDealtToChampions,totalDamageTaken
0,KR_5569285312,1311,11.22.406.3587,마 난다리,591,1,Leblanc,8708,MIDDLE,100,False,4,6,1,7005,12876
1,KR_5569285312,1311,11.22.406.3587,거세폭격기,248,2,Caitlyn,9168,BOTTOM,100,False,4,4,5,8835,10562
2,KR_5569285312,1311,11.22.406.3587,마썽마,250,3,Maokai,7546,UTILITY,100,False,0,5,7,7511,11287
3,KR_5569285312,1311,11.22.406.3587,상어피규어,61,4,Diana,9355,JUNGLE,100,False,6,5,5,10147,18223
4,KR_5569285312,1311,11.22.406.3587,할수있는게머야,259,5,Irelia,7534,TOP,100,False,1,12,0,5014,20875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,KR_5324045449,1626513,11.14.385.9967,나린돼지,389,6,Ryze,12320,TOP,200,True,6,7,8,21319,22184
316,KR_5324045449,1626513,11.14.385.9967,AReflection,207,7,JarvanIV,11565,JUNGLE,200,True,6,6,10,10142,20465
317,KR_5324045449,1626513,11.14.385.9967,Uniq R Xiw,118,8,Irelia,15561,MIDDLE,200,True,13,8,6,28942,33482
318,KR_5324045449,1626513,11.14.385.9967,맨얼굴,420,9,Caitlyn,12532,BOTTOM,200,True,6,2,9,17818,11501


['CLASSIC', 'ARAM']

['MATCHED_GAME']

,gameId,gameDuration,gameVersion,gameMode,summonerName,summonerLevel,participantId,championName,champExperience,teamPosition,teamId,win,kills,deaths,assists,totalDamageDealtToChampions,totalDamageTaken


In [36]:
match_df.head()

,gameId,gameDuration,gameVersion,summonerName,summonerLevel,participantId,championName,champExperience,teamPosition,teamId,win,kills,deaths,assists,totalDamageDealtToChampions,totalDamageTaken
0,KR_5349654495,860490,11.15.388.2387,대한민국남성인권,48,1,Ryze,6645,TOP,100,False,0,5,0,3372,8571
1,KR_5349654495,860490,11.15.388.2387,미드정글유저제로,35,2,LeeSin,4289,JUNGLE,100,False,2,3,1,5010,11006
2,KR_5349654495,860490,11.15.388.2387,Hwanginje,32,3,Yone,3683,MIDDLE,100,False,0,6,1,2229,7875
3,KR_5349654495,860490,11.15.388.2387,모가지 따줄게,33,4,Vayne,3924,BOTTOM,100,False,1,6,0,3805,8599
4,KR_5349654495,860490,11.15.388.2387,정신병자차단함,32,5,Nami,3763,UTILITY,100,False,2,5,1,4332,6738


In [35]:
timeline_df.head()

,gameId,participantId,g_5,g_6,g_7,g_8,g_9,g_10,g_11,g_12,...,g_16,g_17,g_18,g_19,g_20,g_21,g_22,g_23,g_24,g_25
0,KR_5349654495,1,1287,1571,1962,2320,2527,2844,3030,3527,...,0,0,0,0,0,0,0,0,0,0
1,KR_5349654495,2,1333,1850,2266,2488,3508,3654,3838,4236,...,0,0,0,0,0,0,0,0,0,0
2,KR_5349654495,3,1228,1630,1808,1951,2314,2548,2720,2863,...,0,0,0,0,0,0,0,0,0,0
3,KR_5349654495,4,1410,1774,1938,2297,2525,2842,2965,3498,...,0,0,0,0,0,0,0,0,0,0
4,KR_5349654495,5,1226,1381,1575,1749,2057,2237,2457,3298,...,0,0,0,0,0,0,0,0,0,0


In [18]:
mu.db_open()

open!


In [19]:
# match table
query1 = """
create table match_data(gameId varchar(50), gameDuration number(20), gameVersion varchar(50), summonerName varchar(50),
summonerLevel number(20), participantId number(20), championName varchar(50), champExperience number(20),
teamPosition varchar(50), teamId number(20), win varchar(50), kills number(20), deaths number(20), assists number(20),
totalDamageDealtToChampions number(20), totalDamageTaken number(20),
constraint m_pk primary key (gameId, participantId))
"""

In [20]:
# timeline table
query2 = """
create table timeline_data(gameId varchar(50), participantId number(20), g_5 number(20), g_6 number(20), g_7 number(20), g_8 number(20),
g_9 number(20), g_10 number(20), g_11 number(20), g_12 number(20), g_13 number(20), g_14 number(20), g_15 number(20), g_16 number(20),
g_17 number(20), g_18 number(20), g_19 number(20), g_20 number(20), g_21 number(20), g_22 number(20), g_23 number(20), g_24 number(20),
g_25 number(20),
constraint t_pk primary key (gameId, participantId))
"""

In [21]:
res = mu.sql_excute(query1)
res

'success!'

In [22]:
res = mu.sql_excute(query2)
res

'success!'

In [23]:
def match_insert(row):
    query = (
        f'insert into match_data(gameId, gameDuration, gameVersion, summonerName, summonerLevel, participantId, championName, champExperience, '
        f'teamPosition, teamId, win, kills, deaths, assists, totalDamageDealtToChampions, totalDamageTaken) '
        f'values(\'{row.gameId}\', {row.gameDuration}, '
        f'\'{row.gameVersion}\', \'{row.summonerName}\', {row.summonerLevel}, {row.participantId}, '
        f'\'{row.championName}\', {row.champExperience}, \'{row.teamPosition}\', {row.teamId}, '
        f'\'{row.win}\', {row.kills}, {row.deaths}, {row.assists}, '
        f'{row.totalDamageDealtToChampions}, {row.totalDamageTaken}) '        
    )
    mu.sql_excute(query)
    return

In [24]:
def match_insert(row):
    query = (
        f'merge into match_data using dual on(gameId=\'{row.gameId}\' and participantId={row.participantId}) '
        f'when not matched then '
        f'insert (gameId, gameDuration, gameVersion, summonerName, summonerLevel, participantId, championName, champExperience, '
        f'teamPosition, teamId, win, kills, deaths, assists, totalDamageDealtToChampions, totalDamageTaken) '
        f'values(\'{row.gameId}\', {row.gameDuration}, '
        f'\'{row.gameVersion}\', \'{row.summonerName}\', {row.summonerLevel}, {row.participantId}, '
        f'\'{row.championName}\', {row.champExperience}, \'{row.teamPosition}\', {row.teamId}, '
        f'\'{row.win}\', {row.kills}, {row.deaths}, {row.assists}, '
        f'{row.totalDamageDealtToChampions}, {row.totalDamageTaken}) '        
    )
    mu.sql_excute(query)
    return

In [25]:
match_df.progress_apply(lambda x: match_insert(x), axis=1)

100%|███████████████████████████████████████████████████████████████████████████████| 320/320 [00:00<00:00, 517.31it/s]


0      None
1      None
2      None
3      None
4      None
       ... 
315    None
316    None
317    None
318    None
319    None
Length: 320, dtype: object

In [26]:
# mysql 경우 ignore
# insert 할때 중복된 값이 있을경우 insert를 안함
# oralce 경우 merge
# insert 할때 중복된 값 체크

In [27]:
def timeline_insert(row):
    query = (
        f'merge into timeline_data using dual on(gameId=\'{row.gameId}\' and participantId={row.participantId}) '
        f'when not matched then '
        f'insert (gameId, participantId, g_5, g_6, g_7, g_8, g_9, g_10, g_11, g_12, g_13, g_14, g_15, g_16, g_17, g_18, g_19, g_20, '
        f'g_21, g_22, g_23, g_24, g_25) values(\'{row.gameId}\', {row.participantId}, '
        f'{row.g_5}, {row.g_6}, {row.g_7}, {row.g_8}, {row.g_9}, {row.g_10}, {row.g_11}, {row.g_12}, {row.g_13}, {row.g_14}, {row.g_15}, '
        f'{row.g_16}, {row.g_17}, {row.g_18}, {row.g_19}, {row.g_20}, {row.g_21}, {row.g_22}, {row.g_23}, {row.g_24}, {row.g_25})'
    )
    mu.sql_excute(query)
    return

In [28]:
timeline_df.progress_apply(lambda x: timeline_insert(x), axis=1)

100%|███████████████████████████████████████████████████████████████████████████████| 320/320 [00:00<00:00, 491.84it/s]


0      None
1      None
2      None
3      None
4      None
       ... 
315    None
316    None
317    None
318    None
319    None
Length: 320, dtype: object

In [29]:
res = mu.sql_excute('select * from timeline_data')
res

,GAMEID,PARTICIPANTID,G_5,G_6,G_7,G_8,G_9,G_10,G_11,G_12,...,G_16,G_17,G_18,G_19,G_20,G_21,G_22,G_23,G_24,G_25
0,KR_5569285312,1,1270,1857,2084,2297,2569,2817,3127,3249,...,4823,5049,5265,5430,5684,5970,6824,0,0,0
1,KR_5569285312,2,1467,1723,2035,2366,2864,4058,4863,4985,...,6722,7245,7583,7811,8327,8857,9036,0,0,0
2,KR_5569285312,3,1134,1283,1417,1609,1894,2213,2413,2550,...,3362,3513,3779,3922,4239,4518,4863,0,0,0
3,KR_5569285312,4,1489,2283,2590,2812,3392,3799,4210,4332,...,6306,6685,6876,7189,8232,8710,8814,0,0,0
4,KR_5569285312,5,1763,1991,2274,2480,2850,2993,3436,3649,...,4364,4681,4824,5184,5397,5555,6063,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,KR_5324045449,6,1580,2286,2499,2747,3368,3553,3795,3917,...,5767,5964,6250,6463,8002,8617,8968,9131,9589,10011
316,KR_5324045449,7,1471,1894,2196,2583,2806,3159,3432,4064,...,6447,7430,7653,7838,8106,8494,8957,9295,9833,10350
317,KR_5324045449,8,1980,2124,2743,2901,3598,3762,4489,5089,...,6984,7475,7883,8111,8923,9835,10404,10827,11842,12265
318,KR_5324045449,9,1588,2022,2214,2727,3035,3463,3773,4329,...,5966,6879,7240,7393,8381,8886,9621,9848,10058,10781


In [30]:
mu.db_close()

close!


In [31]:
from random import sample
import random
from tqdm import tqdm
import my_utils as mu
import pandas as pd
import requests
tqdm.pandas()

C:\Users\jhkim\anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [32]:
tier = 'GOLD'
raw_data = mu.get_rawData(tier)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

get summonerName...
get puuid...


  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

get matchId...


  0%|                                                                                           | 0/40 [00:00<?, ?it/s]

get match & timeline...


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:38<00:00,  1.03it/s]

complete!


In [33]:
match_df, timeline_df = mu.get_match_timeline_df(raw_data)

 12%|██████████▍                                                                        | 5/40 [00:00<00:00, 40.11it/s]

get matches & timelines...


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 41.18it/s]

match_df length:  280
timeline_df length:  280
complete!


In [34]:
mu.db_open()

open!


In [35]:
def match_insert(row):
    query = (
        f'merge into match_data using dual on(gameId=\'{row.gameId}\' and participantId={row.participantId}) '
        f'when not matched then '
        f'insert (gameId, gameDuration, gameVersion, summonerName, summonerLevel, participantId, championName, champExperience, '
        f'teamPosition, teamId, win, kills, deaths, assists, totalDamageDealtToChampions, totalDamageTaken) '
        f'values(\'{row.gameId}\', {row.gameDuration}, '
        f'\'{row.gameVersion}\', \'{row.summonerName}\', {row.summonerLevel}, {row.participantId}, '
        f'\'{row.championName}\', {row.champExperience}, \'{row.teamPosition}\', {row.teamId}, '
        f'\'{row.win}\', {row.kills}, {row.deaths}, {row.assists}, '
        f'{row.totalDamageDealtToChampions}, {row.totalDamageTaken}) '        
    )
    mu.sql_excute(query)
    return

In [36]:
def timeline_insert(row):
    query = (
        f'merge into timeline_data using dual on(gameId=\'{row.gameId}\' and participantId={row.participantId}) '
        f'when not matched then '
        f'insert (gameId, participantId, g_5, g_6, g_7, g_8, g_9, g_10, g_11, g_12, g_13, g_14, g_15, g_16, g_17, g_18, g_19, g_20, '
        f'g_21, g_22, g_23, g_24, g_25) values(\'{row.gameId}\', {row.participantId}, '
        f'{row.g_5}, {row.g_6}, {row.g_7}, {row.g_8}, {row.g_9}, {row.g_10}, {row.g_11}, {row.g_12}, {row.g_13}, {row.g_14}, {row.g_15}, '
        f'{row.g_16}, {row.g_17}, {row.g_18}, {row.g_19}, {row.g_20}, {row.g_21}, {row.g_22}, {row.g_23}, {row.g_24}, {row.g_25})'
    )
    mu.sql_excute(query)
    return

In [37]:
match_df.progress_apply(lambda x: match_insert(x), axis=1)

100%|███████████████████████████████████████████████████████████████████████████████| 280/280 [00:00<00:00, 520.13it/s]


0      None
1      None
2      None
3      None
4      None
       ... 
275    None
276    None
277    None
278    None
279    None
Length: 280, dtype: object

In [38]:
timeline_df.progress_apply(lambda x: timeline_insert(x), axis=1)

100%|███████████████████████████████████████████████████████████████████████████████| 280/280 [00:00<00:00, 483.05it/s]


0      None
1      None
2      None
3      None
4      None
       ... 
275    None
276    None
277    None
278    None
279    None
Length: 280, dtype: object

In [39]:
res = mu.sql_excute('select * from match_data')
res

,GAMEID,GAMEDURATION,GAMEVERSION,SUMMONERNAME,SUMMONERLEVEL,PARTICIPANTID,CHAMPIONNAME,CHAMPEXPERIENCE,TEAMPOSITION,TEAMID,WIN,KILLS,DEATHS,ASSISTS,TOTALDAMAGEDEALTTOCHAMPIONS,TOTALDAMAGETAKEN
0,KR_5569285312,1311,11.22.406.3587,마 난다리,591,1,Leblanc,8708,MIDDLE,100,False,4,6,1,7005,12876
1,KR_5569285312,1311,11.22.406.3587,거세폭격기,248,2,Caitlyn,9168,BOTTOM,100,False,4,4,5,8835,10562
2,KR_5569285312,1311,11.22.406.3587,마썽마,250,3,Maokai,7546,UTILITY,100,False,0,5,7,7511,11287
3,KR_5569285312,1311,11.22.406.3587,상어피규어,61,4,Diana,9355,JUNGLE,100,False,6,5,5,10147,18223
4,KR_5569285312,1311,11.22.406.3587,할수있는게머야,259,5,Irelia,7534,TOP,100,False,1,12,0,5014,20875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,KR_5495675076,2049,11.20.400.7328,칼부림의 고수,217,6,Sett,17919,TOP,200,False,10,5,6,42396,50408
596,KR_5495675076,2049,11.20.400.7328,다시한번제대로,195,7,Nunu,19653,JUNGLE,200,False,4,4,7,11472,32411
597,KR_5495675076,2049,11.20.400.7328,apokall2,73,8,Akshan,17412,MIDDLE,200,False,7,8,9,37765,29632
598,KR_5495675076,2049,11.20.400.7328,게임하루에1판,181,9,Vayne,13879,BOTTOM,200,False,8,7,3,16666,23770


In [40]:
res = mu.sql_excute('select * from timeline_data')
res

,GAMEID,PARTICIPANTID,G_5,G_6,G_7,G_8,G_9,G_10,G_11,G_12,...,G_16,G_17,G_18,G_19,G_20,G_21,G_22,G_23,G_24,G_25
0,KR_5569285312,1,1270,1857,2084,2297,2569,2817,3127,3249,...,4823,5049,5265,5430,5684,5970,6824,0,0,0
1,KR_5569285312,2,1467,1723,2035,2366,2864,4058,4863,4985,...,6722,7245,7583,7811,8327,8857,9036,0,0,0
2,KR_5569285312,3,1134,1283,1417,1609,1894,2213,2413,2550,...,3362,3513,3779,3922,4239,4518,4863,0,0,0
3,KR_5569285312,4,1489,2283,2590,2812,3392,3799,4210,4332,...,6306,6685,6876,7189,8232,8710,8814,0,0,0
4,KR_5569285312,5,1763,1991,2274,2480,2850,2993,3436,3649,...,4364,4681,4824,5184,5397,5555,6063,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,KR_5495675076,6,1413,1770,1987,2360,2742,3069,3400,3857,...,5457,5642,5980,6118,6310,6600,7181,7908,8031,8826
596,KR_5495675076,7,1549,1797,2121,2333,2671,3461,3784,4408,...,5572,6131,6253,6811,7025,7566,7948,8266,8723,9645
597,KR_5495675076,8,1452,1697,2423,2712,3105,3768,4155,4560,...,5557,5994,6339,6490,7767,8095,8416,8714,9075,9794
598,KR_5495675076,9,1494,1722,1977,2430,2686,2992,3351,3536,...,5523,5673,5816,6958,7080,7482,7924,8493,9020,9237


In [6]:
mu.db_close()

close!


In [41]:
import time

In [42]:
for i in range(50):
    # api 사용하여 raw data 수집
    tier = 'GOLD'
    raw_data = mu.get_rawData(tier)

    # raw data 사용하여 match, timeline dataframe 생성
    match_df, timeline_df = mu.get_match_timeline_df(raw_data)

    # match, timeline dataframe db insert
    mu.db_open()
    match_df.progress_apply(lambda x: match_insert(x), axis=1)
    timeline_df.progress_apply(lambda x: timeline_insert(x), axis=1)
    mu.db_close()
    time.sleep(100)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

get summonerName...
get puuid...


  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

get matchId...


  0%|                                                                                           | 0/38 [00:00<?, ?it/s]

...

complete!
get matches & timelines...


 14%|███████████▌                                                                    | 46/320 [00:00<00:00, 451.24it/s]

match_df length:  320
timeline_df length:  320
complete!
open!


100%|███████████████████████████████████████████████████████████████████████████████| 320/320 [00:00<00:00, 469.72it/s]


close!


In [43]:
mu.db_open()
df = mu.sql_excute('select * from match_data')
mu.db_close()
df

open!
close!


,GAMEID,GAMEDURATION,GAMEVERSION,SUMMONERNAME,SUMMONERLEVEL,PARTICIPANTID,CHAMPIONNAME,CHAMPEXPERIENCE,TEAMPOSITION,TEAMID,WIN,KILLS,DEATHS,ASSISTS,TOTALDAMAGEDEALTTOCHAMPIONS,TOTALDAMAGETAKEN
0,KR_5569285312,1311,11.22.406.3587,마 난다리,591,1,Leblanc,8708,MIDDLE,100,False,4,6,1,7005,12876
1,KR_5569285312,1311,11.22.406.3587,거세폭격기,248,2,Caitlyn,9168,BOTTOM,100,False,4,4,5,8835,10562
2,KR_5569285312,1311,11.22.406.3587,마썽마,250,3,Maokai,7546,UTILITY,100,False,0,5,7,7511,11287
3,KR_5569285312,1311,11.22.406.3587,상어피규어,61,4,Diana,9355,JUNGLE,100,False,6,5,5,10147,18223
4,KR_5569285312,1311,11.22.406.3587,할수있는게머야,259,5,Irelia,7534,TOP,100,False,1,12,0,5014,20875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1805,KR_5408631267,1304555,11.16.390.1945,쿠모리,273,6,Rengar,13001,TOP,200,True,9,2,5,17758,22058
1806,KR_5408631267,1304555,11.16.390.1945,노력파두목,458,7,Ekko,11221,JUNGLE,200,True,9,5,12,16881,21575
1807,KR_5408631267,1304555,11.16.390.1945,쌉똥쟁이,164,8,AurelionSol,10445,MIDDLE,200,True,4,4,9,10281,10943
1808,KR_5408631267,1304555,11.16.390.1945,핵똥쟁이,264,9,Caitlyn,9605,BOTTOM,200,True,10,1,10,16976,7847
